In [62]:
from collections import defaultdict

def create_parent(node_dict, key, avail_vars, input_vars: dict, idx):
    content = node_dict[key]["content"]
    parent_content = []

    #Input nodes
    if not isinstance(content, list) == 1: #If we arrive at the end of unpacking
        if content in input_vars:
            node_dict[key]["value"] = input_vars[content] #Setting value to input
        else:
            node_dict[key]["value"] = content
        return node_dict, []

    #Operation nodes
    for i, cont in enumerate(content):
        if cont in ["+", "-", "*", "/", "^"]: #bivariate function
            parent_content.extend(content[:i])
            parent_content.extend(content[i+1:])
            operation = cont
            break
        elif cont in ["exp", "log", "sin", "cos"]: #univariate functions 
            parent_content = [content[1]]
            operation = cont
            break
        else:
            continue


    #Names for new keys, have to check if part of content is an input variable
    new_keys = []
    input_vars = list(input_vars.keys()) 
    for i, cont in enumerate(parent_content):
        if cont in input_vars: #If it is an input variable, key name = variable name
            new_keys.append(cont)
        else:
            new_keys.append(avail_vars[idx + i]) #If it is not an input, give it a name from the list

    #Adding parent keys, parent content and operation on the parents which is performed in current node
    node_dict[key]["parents"] = new_keys
    node_dict[key]["parent_contents"] = parent_content
    node_dict[key]["operator"] = operation

    #Creating parent nodes, adding their content and children
    for i, new_key in enumerate(new_keys):
        node = defaultdict()
        node["content"] = parent_content[i]
        node["children"] = [key]
        #Adding the created node as entry to the dict
        node_dict[new_key] = node
        #print("key: ", new_key, " | content", node["content"])
    

    #Return Dict and new_nodes
    return node_dict, new_keys


class Builder():

    def __init__(self, infix: list, in_vars: dict = {}):
        """
        infix: list of infix notation parse, e.g. [['exp', 2], '-', 3]
        in_vars: dict of input variables to ensure they are not used as intermediate or output variables
        RETURN: computation graph in a data structure of your choosing
        """

        ## some alphabetical vars to use as intermediate and output variables minus the input vars to avoid duplicates
        avail_vars = list(map(chr, range(97, 123))) + list(map(chr, range(945, 969)))
        if len(in_vars.keys()) > 0:
            avail_vars = set(avail_vars) - set(in_vars)
        self.avail_vars = sorted(list(set(avail_vars)), reverse=True)
        self.in_vars = in_vars
        self.infix = infix
        # Making a dictionary. The key is the node name chosen from avail_vars. Value is another dictionary with keys:
        # Parents: keynames of the parent nodes, set to None if input nodes
        # Children: keynames of the child nodes, set to None if output nodes
        # Value: Is value of node resulting from forward pass. Beginning None, will be filled when forward pass is initialized. 
        # Operator: Operator sign, to map node to an operator class in the forward pass
        # Content: Is the infix content of the node, as sanity check
        keys = ["output"]
        node_dict = defaultdict(None, {"output": {"content": infix, "children": None}})
        while len(keys) > 0:
            key = keys[0]
            node_dict, new_keys = create_parent(node_dict, key = key, avail_vars = self.avail_vars, input_vars=self.in_vars, idx = len(node_dict.keys())-1)
            keys.extend(new_keys) #appending keys to key list
            keys = keys[1:] #removing key I just worked on from new keys
        self.graph = node_dict

#infix = ['z', '+', ['sin', [['x', '^', 2], '+', ['y', '*', ['exp', 'z']]]]]
infix = [[['x', '^', 2], '-', 1], '*', ['y', '+', 2]]
#infix = ["x", "/", "y"]
in_vars = {"x":1, "y": 5}
builder = Builder(infix, in_vars)


In [63]:
from abc import ABC, abstractmethod
import math

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]


class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return math.exp(a)


class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        return math.log(a)

    def df(self, a, b = None):
        return 1/a


class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]


class Div(Operator):

    def f(self, a, b):
        return a/b

    def df(self, a, b):
        return [1/b, a/(b**2)]

class Add(Operator):

    def f(self, a, b):
        return a+b

    def df(self, a, b = None):
        return [1,1]


class Sub(Operator):

    def f(self, a, b = None):
        return a-b

    def df(self, a, b = None):
        return [1, -1]


class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]


class Sin(Operator):

    def f(self, a, b=None):
        return math.sin(a)

    def df(self, a, b=None):
        return math.cos(a)


class Cos(Operator):

    def f(self, a, b=None):
        return math.cos(a)

    def df(self, a, b=None):
        return -math.sin(a)



In [69]:
class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self):
        graph = self.graph
        in_vars = self.in_vars
        map_dict = self.fn_map
        starter_nodes = []

        # 1 go over all nodes by iterating over their keys
        # 2 check if a key has a value already, if it does remove it from current node list
        # 3 if the key has no value check if all its parents have values. If not continue with loop
        # 4 If all parents have a value continue to do operation
        current_nodes = list(graph.keys())
        i = 0
        while graph["output"].get("value") == None:
            print("remaining nodes: ", current_nodes)
            for key in current_nodes:
                print("currently working on key: ", key)
                if graph[key].get("value") != None: 
                    current_nodes.remove(key)
                    print("key was removed from list")
                    #If they already have values, forward doesn't need to work on those nodes anymor
                else: #All nodes that don't already have values
                    operation = graph[key].get("operator")
                    operator = map_dict[operation]
                    parents = graph[key].get("parents") #Look for parents of nodes
                    parent_values = []
                    print("parents: ", parents)
                    for parent in parents: #Make a list of the parents values
                        parent_values.append(graph[parent].get("value"))
                    print("parent values: ", parent_values)
                    if None not in parent_values: #If all parents have arguments, you can do operation
                        a = parent_values[0]
                        b = None
                        if len(parents) == 2: #Only for bivariate functions
                            b = parent_values[1]
                        graph[key]["value"] = operator.f(a,b)
                        print("all parents had values, node value: ", graph[key]["value"])
                        current_nodes.remove(key)
                        
                    else: #If some parents don't have a value yet
                        print("Not all parents had values, continuing")
                        
            print("started list over")
        print("Output created")
        self.output = graph["output"].get("value")

    ## backward execution____________________________

    def backward(self, ):
        #Initializing Variables
        graph = self.graph
        in_vars = self.in_vars
        map_dict = self.fn_map

        #Initializing input
        graph["output"]["df"] = 1
        current_nodes = ["output"]

        #Creating df for nodes
        while len(current_nodes) >0:
            parent_list = []

            #Filling out parent df
            for key in current_nodes:
                #Check if this is an input node
                parents = graph[key].get("parents")
                if parents == None:
                    continue

                #Find the operation done in the current node
                operation = graph[key]["operator"]
                operator = map_dict[operation]

                #Finding parent arguments and assign the df value:
                parents = graph[key]["parents"]
                df_before = graph[key]["df"] #Is the derivative of current before

                if len(parents) == 1:
                    parent_a = parents[0] #Parent node, that supplied argument a
                    a = graph[parent_a]["value"]
                    graph[parent_a]["df"] = operator.df(a)*df_before
                    
                elif len(parents) == 2:
                    parent_a = parents[0]
                    parent_b = parents[1]
                    a = graph[parent_a]["value"]
                    b = graph[parent_b]["value"]
                    graph[parent_a]["df"], graph[parent_b]["df"] = [df*df_before for df in operator.df(a,b)]
                #Appending parent keys, to parents list for next layer
                parent_list.extend(parents)

            #Initializing nodes for next layer
            current_nodes = parent_list

        #Output dictionary of input vars with their df
        self.derivative = {}
        for key in list(in_vars.keys()):
            self.derivative[key] = graph[key]["df"]
        print("Derivatives Created")

exec = Executor(builder.graph, builder.in_vars)
exec.forward()
exec.backward()
print("Output: ", exec.output, "\nDerivatives; ", exec.derivative)

Output created
Derivatives Created
Output:  0 
Derivatives;  {'x': 14, 'y': 0}


In [34]:
0 != None

True